In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.db as db
import re
import pandas as pd
import pathlib

# Aim

Remove the images that were manually determined bad from the valid images

In [ ]:
class V:
    COL_ISBAD = 'is_bad'
class C:
    fn_config = sm.input.fn_config
    fn_manual_qc_dat = sm.input.fn_manual_qc_dat

In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())
print(n_img, n_obj)


In [ ]:
dat_qc = pd.read_csv(C.fn_manual_qc_dat)
dat_qc[V.COL_ISBAD] = True 

In [ ]:
q = (bro.session.query(db.images.acquisition_id,
                     db.images.image_id,
                     db.images.image_number,
                     db.images.crop_number,
                     db.acquisitions.acquisition_mcd_acid,
                     db.sites.site_mcd_panoramaid,
                     db.slideacs.slideac_name)
     .join(db.acquisitions, db.images.acquisition_id == db.acquisitions.acquisition_id)
     .join(db.sites)
     .join(db.slideacs)
    )
dat_imgs = bro.doquery(q)

In [ ]:
dat_imgs.query('image_number == 501')

In [ ]:
dat_isbad = dat_imgs.merge(dat_qc[[db.images.crop_number.key,
                                db.acquisitions.acquisition_mcd_acid.key,
                                db.slideacs.slideac_name.key]])

In [ ]:
dat_isbad.head()

In [ ]:
bad_img_ids = list(dat_isbad[db.images.image_id.key])

In [ ]:
stmt = (bro.session.query(db.valid_images)
 .filter(db.valid_images.image_id.in_(bad_img_ids)))

print(f'{len(bad_img_ids)} determined bad of which {stmt.count()} were in the valid images')

Do the deletion

In [ ]:
stmt.delete(synchronize_session='fetch')
bro.session.commit()

Sanity check - plot all the 'bad' images:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for im in bad_img_ids:
    fig = bro.plots.heatmask.plt_heatplot([im], stat='MeanIntensityComp', stack='FullStackFiltered', channel='Ir193',
                                    title=f'image {im}', valid_images=False
                               )
    plt.show()
    plt.close()